In [1]:
#!/usr/bin/env python
# coding: utf-8


#libraries

import pandas as pd
import numpy  as np 
import pickle
import joblib
import http.client
import json
import time
import csv
from   datetime import datetime, timedelta
import holidays
import sqlite3
import re
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

# Configure Nrgstrem API call functionalities

class NRGStreamApi:    
    
    def __init__(self,username=None,password=None):
        self.username = 'Fahad'
        self.password = 'ABFAHc2'                
        self.server = 'api.nrgstream.com'        
        self.tokenPath = '/api/security/token'
        self.releasePath = '/api/ReleaseToken'
        self.tokenPayload = f'grant_type=password&username={self.username}&password={self.password}'
        self.tokenExpiry = datetime.now() - timedelta(seconds=60)
        self.accessToken = ""        
        
    def getToken(self):
        try:
            if self.isTokenValid() == False:                             
                headers = { }        
                # Connect to API server to get a token
                conn = http.client.HTTPSConnection(self.server)
                conn.request('POST', self.tokenPath, self.tokenPayload, headers)
                res = conn.getresponse()                
                res_code = res.code
                # Check if the response is good
                
                if res_code == 200:
                    res_data = res.read()
                    # Decode the token into an object
                    jsonData = json.loads(res_data.decode('utf-8'))
                    self.accessToken = jsonData['access_token']                         
                    # Calculate new expiry date
                    self.tokenExpiry = datetime.now() + timedelta(seconds=jsonData['expires_in'])                        
                    #print('token obtained')
                    #print(self.accessToken)
                else:
                    res_data = res.read()
                    print(res_data.decode('utf-8'))
                conn.close()                          
        except Exception as e:
            print("getToken: " + str(e))
            # Release token if an error occured
            self.releaseToken()      

    def releaseToken(self):
        try:            
            headers = {}
            headers['Authorization'] = f'Bearer {self.accessToken}'            
            conn = http.client.HTTPSConnection(self.server)
            conn.request('DELETE', self.releasePath, None, headers)  
            res = conn.getresponse()
            res_code = res.code
            if res_code == 200:   
                # Set expiration date back to guarantee isTokenValid() returns false                
                self.tokenExpiry = datetime.now() - timedelta(seconds=60)
                #print('token released')            
        except Exception as e:
            print("releaseToken: " + str(e))
                    
    def isTokenValid(self):
        if self.tokenExpiry==None:
            return False
        elif datetime.now() >= self.tokenExpiry:            
            return False
        else:
            return True            
    
    def GetStreamDataByStreamId(self,streamIds, fromDate, toDate, dataFormat='csv', dataOption=''):
        stream_data = "" 
        # Set file format to csv or json            
        DataFormats = {}
        DataFormats['csv'] = 'text/csv'
        DataFormats['json'] = 'Application/json'
        
        try:                            
            for streamId in streamIds:            
                # Get an access token            
                self.getToken()    
                if self.isTokenValid():
                    # Setup the path for data request. Pass dates in via function call
                    path = f'/api/StreamData/{streamId}'
                    if fromDate != '' and toDate != '':
                        path += f'?fromDate={fromDate.replace(" ", "%20")}&toDate={toDate.replace(" ", "%20")}'
                    if dataOption != '':
                        if fromDate != '' and toDate != '':
                            path += f'&dataOption={dataOption}'        
                        else:
                            path += f'?dataOption={dataOption}'        
                    
                    # Create request header
                    headers = {}            
                    headers['Accept'] = DataFormats[dataFormat]
                    headers['Authorization']= f'Bearer {self.accessToken}'
                    
                    # Connect to API server
                    conn = http.client.HTTPSConnection(self.server)
                    conn.request('GET', path, None, headers)
                    res = conn.getresponse()        
                    res_code = res.code                    
                    if res_code == 200:   
                        try:
                            print(f'{datetime.now()} Outputing stream {path} res code {res_code}')
                            # output return data to a text file            
                            if dataFormat == 'csv':
                                stream_data += res.read().decode('utf-8').replace('\r\n','\n') 
                            elif dataFormat == 'json':
                                stream_data += json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)
                            conn.close()

                        except Exception as e:
                            print(str(e))            
                            self.releaseToken()
                            return None  
                    else:
                        print(str(res_code) + " - " + str(res.reason) + " - " + str(res.read().decode('utf-8')))
                    
                self.releaseToken()   
                # Wait 1 second before next request
                time.sleep(1)
            return stream_data        
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None
        
        
    def StreamDataOptions(self, streamId, dataFormat='csv'):
        try:      
            DataFormats = {}
            DataFormats['csv'] = 'text/csv'
            DataFormats['json'] = 'Application/json'
            resultSet = {}
            for streamId in streamIds:
                # Get an access token    
                if streamId not in resultSet:
                    self.getToken()                        
                    if self.isTokenValid():                 
                        # Setup the path for data request.
                        path = f'/api/StreamDataOptions/{streamId}'                        
                        # Create request header
                        headers = {}     
                        headers['Accept'] = DataFormats[dataFormat]                                   
                        headers['Authorization'] = f'Bearer {self.accessToken}'
                        # Connect to API server
                        conn = http.client.HTTPSConnection(self.server)
                        conn.request('GET', path, None, headers)
                        res = conn.getresponse()
                        self.releaseToken()       
                        if dataFormat == 'csv':
                            resultSet[streamId] = res.read().decode('utf-8').replace('\r\n','\n') 
                        elif dataFormat == 'json':
                            resultSet[streamId] = json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)                            
                    time.sleep(1)                        
            return resultSet            
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None          
        
        except Exception as e:            
            self.releaseToken()                        
            return str(e)        

# Authenticate with your NRG Stream username and password    
nrgStreamApi = NRGStreamApi('Username','Password')      

# in-memory weather database loading- deleted

#conn = sqlite3.connect('weather_db.db')
#c = conn.cursor()
#c.execute('''  
#SELECT * FROM HISTORICALFCAST
#          ''')
#results = c.fetchall()
#weather_df = pd.DataFrame(results)

#c.close()
#conn.close()

# generate time-stamps for forecast hours

now_time                 = datetime.now()
year_of_run              = now_time.year
month_of_run             = now_time.month
day_of_run               = now_time.day
forecast_start           = datetime(year_of_run ,month_of_run , day_of_run)
forecast_interval_stamps = pd.date_range(forecast_start, periods=24, freq= 'h')

# create the dataframe to hold input data for 24-h forecast

column_list = ['hour_of_day',  'day','hour_x_day',  'weekend', 'month', 'year',
       'AIL_24h_lagged', 'AIL_2day_lagged',
       'AIL_3day_lagged', 'AIL_4day_lagged', 'AIL_5day_lagged',
       'AIL_6day_lagged', 'AIL_oneweek_lagged', 
       'temp_calgary','temp_edmonton', 'temp_calgary_lag1',
       'temp_calgary_lag2', 'temp_calgary_lag3', 'temp_calgary_lag1_squared',
       'temp_calgary_lag2_squared', 'temp_calgary_lag3_squared',
       'temp_calgary_lag1_cube', 'temp_calgary_lag2_cube',
       'temp_calgary_lag3_cube', 'temp_edmonton_lag1', 'temp_edmonton_lag2',
       'temp_edmonton_lag3', 'temp_edmonton_lag1_squared',
       'temp_edmonton_lag2_squared', 'temp_edmonton_lag3_squared',
       'temp_edmonton_lag1_cube', 'temp_edmonton_lag2_cube',
       'temp_edmonton_lag3_cube', 'hour_temp_calgary', 'month_temp_calgary',
       'hour_temp_calgary_lag1', 'hour_temp_calgary_lag2',
       'hour_temp_calgary_lag3', 'month_temp_calgary_lag1',
       'month_temp_calgary_lag2', 'month_temp_calgary_lag3',
       'hour_temp_calgary_lag1_sqd', 'hour_temp_calgary_lag2_sqd',
       'hour_temp_calgary_lag3_sqd', 'month_temp_calgary_lag1_cube',
       'month_temp_calgary_lag2_cube', 'month_temp_calgary_lag3_cube',
       'hour_temp_edmonton', 'month_temp_edmonton', 'hour_temp_edmonton_lag1',
       'hour_temp_edmonton_lag2', 'hour_temp_edmonton_lag3',
       'month_temp_edmonton_lag1', 'month_temp_edmonton_lag2',
       'month_temp_edmonton_lag3', 'hour_temp_edmonton_lag1_sqd',
       'hour_temp_edmonton_lag2_sqd', 'hour_temp_edmonton_lag3_sqd',
       'month_temp_edmonton_lag1_cube', 'month_temp_edmonton_lag2_cube',
       'month_temp_edmonton_lag3_cube' ]



prediction_df            = pd.DataFrame(columns = column_list)


###########################################################################
####################### Feature Generation ################################
###########################################################################

hrs                          = np.arange(0, 24, 1).tolist()
prediction_df['hour_of_day'] = hrs

# add time-related features

prediction_df['day']       = forecast_interval_stamps.day
prediction_df['hour_x_day'] = prediction_df['day']*prediction_df['hour_of_day'] 
weekdays                  = [d.weekday() for d in forecast_interval_stamps]
prediction_df['weekend']  = [1 if d >= 5 else 0 for d in weekdays]
prediction_df['month'] = forecast_interval_stamps.month
prediction_df['year'] = year_of_run

In [2]:
for i in range(0,len(prediction_df.columns)):
    print(i, '--->',prediction_df.columns[i],'--->', prediction_df.dtypes[i])

0 ---> hour_of_day ---> int64
1 ---> day ---> int64
2 ---> hour_x_day ---> int64
3 ---> weekend ---> int64
4 ---> month ---> int64
5 ---> year ---> int64
6 ---> AIL_24h_lagged ---> object
7 ---> AIL_2day_lagged ---> object
8 ---> AIL_3day_lagged ---> object
9 ---> AIL_4day_lagged ---> object
10 ---> AIL_5day_lagged ---> object
11 ---> AIL_6day_lagged ---> object
12 ---> AIL_oneweek_lagged ---> object
13 ---> temp_calgary ---> object
14 ---> temp_edmonton ---> object
15 ---> temp_calgary_lag1 ---> object
16 ---> temp_calgary_lag2 ---> object
17 ---> temp_calgary_lag3 ---> object
18 ---> temp_calgary_lag1_squared ---> object
19 ---> temp_calgary_lag2_squared ---> object
20 ---> temp_calgary_lag3_squared ---> object
21 ---> temp_calgary_lag1_cube ---> object
22 ---> temp_calgary_lag2_cube ---> object
23 ---> temp_calgary_lag3_cube ---> object
24 ---> temp_edmonton_lag1 ---> object
25 ---> temp_edmonton_lag2 ---> object
26 ---> temp_edmonton_lag3 ---> object
27 ---> temp_edmonton_lag1_squa

In [3]:
#one-day in the past feature
datetime_pastday                = now_time + pd.Timedelta(days= -1)
fromDate                        = '{}/{}/{}'.format(datetime_pastday.month,datetime_pastday.day, datetime_pastday.year )
toDate                          = fromDate
stream_data                     = nrgStreamApi.GetStreamDataByStreamId([3], fromDate, toDate, 'csv', '')        
STREAM_DATA                     = StringIO(stream_data)
stream_df_yesterday             = pd.read_csv(STREAM_DATA, sep=";")
stream_df_yesterday             = stream_df_yesterday.iloc[14:38,:]
stream_df_yesterday.columns     = ["Datetime,AIL"]
temp_df                         = stream_df_yesterday['Datetime,AIL'].str.split(",", n = 2, expand = True) 
stream_df_yesterday["Datetime"] = temp_df[0] 
stream_df_yesterday['AIL']      = temp_df[1] 
stream_df_yesterday['AIL']      = pd.to_numeric(stream_df_yesterday['AIL'],errors='coerce')
stream_df_yesterday             = stream_df_yesterday.drop(columns=['Datetime,AIL','Datetime'],axis=1)
prediction_df['AIL_24h_lagged'] = stream_df_yesterday.values
prediction_df['AIL_24h_lagged'] = prediction_df['AIL_24h_lagged'].astype('float64')

#2-day in the past feature
datetime_pastday                = now_time + pd.Timedelta(days= -2)
fromDate                        = '{}/{}/{}'.format(datetime_pastday.month,datetime_pastday.day, datetime_pastday.year )
toDate                          = fromDate
stream_data                     = nrgStreamApi.GetStreamDataByStreamId([3], fromDate, toDate, 'csv', '')        
STREAM_DATA                     = StringIO(stream_data)
stream_df_yesterday             = pd.read_csv(STREAM_DATA, sep=";")
stream_df_yesterday             = stream_df_yesterday.iloc[14:38,:]
stream_df_yesterday.columns     = ["Datetime,AIL"]
temp_df                         = stream_df_yesterday['Datetime,AIL'].str.split(",", n = 2, expand = True) 
stream_df_yesterday["Datetime"] = temp_df[0] 
stream_df_yesterday['AIL']      = temp_df[1] 
stream_df_yesterday['AIL']      = pd.to_numeric(stream_df_yesterday['AIL'],errors='coerce')
stream_df_yesterday             = stream_df_yesterday.drop(columns=['Datetime,AIL','Datetime'],axis=1)
prediction_df['AIL_2day_lagged'] = stream_df_yesterday.values
prediction_df['AIL_2day_lagged'] = prediction_df['AIL_2day_lagged'].astype('float64')

#3-day in the past feature
datetime_pastday                = now_time + pd.Timedelta(days= -3)
fromDate                        = '{}/{}/{}'.format(datetime_pastday.month,datetime_pastday.day, datetime_pastday.year )
toDate                          = fromDate
stream_data                     = nrgStreamApi.GetStreamDataByStreamId([3], fromDate, toDate, 'csv', '')        
STREAM_DATA                     = StringIO(stream_data)
stream_df_yesterday             = pd.read_csv(STREAM_DATA, sep=";")
stream_df_yesterday             = stream_df_yesterday.iloc[14:38,:]
stream_df_yesterday.columns     = ["Datetime,AIL"]
temp_df                         = stream_df_yesterday['Datetime,AIL'].str.split(",", n = 2, expand = True) 
stream_df_yesterday["Datetime"] = temp_df[0] 
stream_df_yesterday['AIL']      = temp_df[1] 
stream_df_yesterday['AIL']      = pd.to_numeric(stream_df_yesterday['AIL'],errors='coerce')
stream_df_yesterday             = stream_df_yesterday.drop(columns=['Datetime,AIL','Datetime'],axis=1)
prediction_df['AIL_3day_lagged'] = stream_df_yesterday.values
prediction_df['AIL_3day_lagged'] = prediction_df['AIL_3day_lagged'].astype('float64')

#4-day in the past feature
datetime_pastday                = now_time + pd.Timedelta(days= -4)
fromDate                        = '{}/{}/{}'.format(datetime_pastday.month,datetime_pastday.day, datetime_pastday.year )
toDate                          = fromDate
stream_data                     = nrgStreamApi.GetStreamDataByStreamId([3], fromDate, toDate, 'csv', '')        
STREAM_DATA                     = StringIO(stream_data)
stream_df_yesterday             = pd.read_csv(STREAM_DATA, sep=";")
stream_df_yesterday             = stream_df_yesterday.iloc[14:38,:]
stream_df_yesterday.columns     = ["Datetime,AIL"]
temp_df                         = stream_df_yesterday['Datetime,AIL'].str.split(",", n = 2, expand = True) 
stream_df_yesterday["Datetime"] = temp_df[0] 
stream_df_yesterday['AIL']      = temp_df[1] 
stream_df_yesterday['AIL']      = pd.to_numeric(stream_df_yesterday['AIL'],errors='coerce')
stream_df_yesterday             = stream_df_yesterday.drop(columns=['Datetime,AIL','Datetime'],axis=1)
prediction_df['AIL_4day_lagged'] = stream_df_yesterday.values
prediction_df['AIL_4day_lagged'] = prediction_df['AIL_4day_lagged'].astype('float64')

#5-day in the past feature
datetime_pastday                = now_time + pd.Timedelta(days= -5)
fromDate                        = '{}/{}/{}'.format(datetime_pastday.month,datetime_pastday.day, datetime_pastday.year )
toDate                          = fromDate
stream_data                     = nrgStreamApi.GetStreamDataByStreamId([3], fromDate, toDate, 'csv', '')        
STREAM_DATA                     = StringIO(stream_data)
stream_df_yesterday             = pd.read_csv(STREAM_DATA, sep=";")
stream_df_yesterday             = stream_df_yesterday.iloc[14:38,:]
stream_df_yesterday.columns     = ["Datetime,AIL"]
temp_df                         = stream_df_yesterday['Datetime,AIL'].str.split(",", n = 2, expand = True) 
stream_df_yesterday["Datetime"] = temp_df[0] 
stream_df_yesterday['AIL']      = temp_df[1] 
stream_df_yesterday['AIL']      = pd.to_numeric(stream_df_yesterday['AIL'],errors='coerce')
stream_df_yesterday             = stream_df_yesterday.drop(columns=['Datetime,AIL','Datetime'],axis=1)
prediction_df['AIL_5day_lagged'] = stream_df_yesterday.values
prediction_df['AIL_5day_lagged'] = prediction_df['AIL_5day_lagged'].astype('float64')

#6-day in the past feature
datetime_pastday                = now_time + pd.Timedelta(days= -6)
fromDate                        = '{}/{}/{}'.format(datetime_pastday.month,datetime_pastday.day, datetime_pastday.year )
toDate                          = fromDate
stream_data                     = nrgStreamApi.GetStreamDataByStreamId([3], fromDate, toDate, 'csv', '')        
STREAM_DATA                     = StringIO(stream_data)
stream_df_yesterday             = pd.read_csv(STREAM_DATA, sep=";")
stream_df_yesterday             = stream_df_yesterday.iloc[14:38,:]
stream_df_yesterday.columns     = ["Datetime,AIL"]
temp_df                         = stream_df_yesterday['Datetime,AIL'].str.split(",", n = 2, expand = True) 
stream_df_yesterday["Datetime"] = temp_df[0] 
stream_df_yesterday['AIL']      = temp_df[1] 
stream_df_yesterday['AIL']      = pd.to_numeric(stream_df_yesterday['AIL'],errors='coerce')
stream_df_yesterday             = stream_df_yesterday.drop(columns=['Datetime,AIL','Datetime'],axis=1)
prediction_df['AIL_6day_lagged'] = stream_df_yesterday.values
prediction_df['AIL_6day_lagged'] = prediction_df['AIL_6day_lagged'].astype('float64')


# 7-day in the past feature
datetime_pastweek               = now_time + pd.Timedelta(days= -7)
fromDate                        = '{}/{}/{}'.format(datetime_pastweek.month ,datetime_pastweek.day, datetime_pastweek.year)
toDate                          = fromDate
stream_data                     = nrgStreamApi.GetStreamDataByStreamId([3], fromDate, toDate, 'csv', '')        
STREAM_DATA                     = StringIO(stream_data)
stream_df_7daypast              = pd.read_csv(STREAM_DATA, sep=";")
stream_df_7daypast              = stream_df_7daypast.iloc[14:38,:]
stream_df_7daypast.columns      = ["Datetime,AIL"]
temp_df                         = stream_df_7daypast['Datetime,AIL'].str.split(",", n = 2, expand = True) 
stream_df_7daypast["Datetime"]  = temp_df[0] 
stream_df_7daypast['AIL']       = temp_df[1] 
stream_df_7daypast['AIL']       = pd.to_numeric(stream_df_7daypast['AIL'],errors='coerce')
stream_df_7daypast              = stream_df_7daypast.drop(columns=['Datetime,AIL','Datetime'],axis=1)
prediction_df['AIL_oneweek_lagged'] = stream_df_7daypast.values
prediction_df['AIL_oneweek_lagged'] = prediction_df['AIL_oneweek_lagged'].astype('float64')

2021-02-16 10:57:48.876540 Outputing stream /api/StreamData/3?fromDate=2/15/2021&toDate=2/15/2021 res code 200
2021-02-16 10:57:50.411310 Outputing stream /api/StreamData/3?fromDate=2/14/2021&toDate=2/14/2021 res code 200
2021-02-16 10:57:51.939486 Outputing stream /api/StreamData/3?fromDate=2/13/2021&toDate=2/13/2021 res code 200
2021-02-16 10:57:54.618762 Outputing stream /api/StreamData/3?fromDate=2/12/2021&toDate=2/12/2021 res code 200
2021-02-16 10:57:57.032811 Outputing stream /api/StreamData/3?fromDate=2/11/2021&toDate=2/11/2021 res code 200
2021-02-16 10:57:59.655342 Outputing stream /api/StreamData/3?fromDate=2/10/2021&toDate=2/10/2021 res code 200
2021-02-16 10:58:01.223985 Outputing stream /api/StreamData/3?fromDate=2/9/2021&toDate=2/9/2021 res code 200


In [4]:
# weather feature

fromDate = '{}/{}/{}'.format(month_of_run,day_of_run, year_of_run )
toDate   = fromDate

df_clgry   = pd.DataFrame()
df_edmtn   = pd.DataFrame()
df_list = {}

streams = [242498, 242497]
city_names = ['calgary', 'edmonton']
k = 0

for i in streams:
    stream_data = nrgStreamApi.GetStreamDataByStreamId([i], fromDate, toDate, 'csv', '')        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")
    
    df = df[16:(16+24)] # removing header information
    df.columns = ["Datetime,temp,wind,direction"]
    new = df['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True) 
    # making separate first name column from new data frame 
    df["Datetime"]= new[0] 
    df["temp"]= new[1] 
    df["wind"] = new [2]
    df["direction"] = new[3]
    df = df.drop(['Datetime,temp,wind,direction','direction','Datetime', 'wind'],axis=1)
    df = df.reset_index(drop=True)
    df.columns = [str(col) + '_'+ city_names[k] for col in df.columns]
    df_list[k] = df
    k = k+1

df_clgry   = df_list[0]
df_edmtn   = df_list[1]

alberta_weather_merged = pd.concat([df_clgry, df_edmtn], axis=1)
alberta_weather_merged

2021-02-16 10:58:02.741378 Outputing stream /api/StreamData/242498?fromDate=2/16/2021&toDate=2/16/2021 res code 200
2021-02-16 10:58:04.228982 Outputing stream /api/StreamData/242497?fromDate=2/16/2021&toDate=2/16/2021 res code 200


,temp_calgary,temp_edmonton
0,-20,-20
1,-20,-20
2,-19,-20
3,-18,-20
4,-17,-20
5,-17,-22
6,-15,-21
7,-14,-21
8,-12,-20
9,-10,-19


In [5]:
alberta_weather_merged['temp_calgary_lag1'] = alberta_weather_merged['temp_calgary'].shift(1)
alberta_weather_merged['temp_calgary_lag2'] = alberta_weather_merged['temp_calgary'].shift(2)
alberta_weather_merged['temp_calgary_lag3'] = alberta_weather_merged['temp_calgary'].shift(3)

alberta_weather_merged['temp_edmonton_lag1'] = alberta_weather_merged['temp_edmonton'].shift(1)
alberta_weather_merged['temp_edmonton_lag2'] = alberta_weather_merged['temp_edmonton'].shift(2)
alberta_weather_merged['temp_edmonton_lag3'] = alberta_weather_merged['temp_edmonton'].shift(3)
alberta_weather_merged

,temp_calgary,temp_edmonton,temp_calgary_lag1,temp_calgary_lag2,temp_calgary_lag3,temp_edmonton_lag1,temp_edmonton_lag2,temp_edmonton_lag3
0,-20,-20,NaN,NaN,NaN,NaN,NaN,NaN
1,-20,-20,-20,NaN,NaN,-20,NaN,NaN
2,-19,-20,-20,-20,NaN,-20,-20,NaN
3,-18,-20,-19,-20,-20,-20,-20,-20
4,-17,-20,-18,-19,-20,-20,-20,-20
5,-17,-22,-17,-18,-19,-20,-20,-20
6,-15,-21,-17,-17,-18,-22,-20,-20
7,-14,-21,-15,-17,-17,-21,-22,-20
8,-12,-20,-14,-15,-17,-21,-21,-22
9,-10,-19,-12,-14,-15,-20,-21,-21


In [6]:
#one-day in the past feature
datetime_pastday                = now_time + pd.Timedelta(days= -1)
fromDate                        = '{}/{}/{}'.format(datetime_pastday.month,datetime_pastday.day, datetime_pastday.year )
toDate                          = fromDate
stream_data                     = nrgStreamApi.GetStreamDataByStreamId([242498], fromDate, toDate, 'csv', '')        
STREAM_DATA                     = StringIO(stream_data)
stream_df_yesterday             = pd.read_csv(STREAM_DATA, sep=";")
stream_df_yesterday             = stream_df_yesterday.iloc[16:]
stream_df_yesterday.columns     = ["Datetime,temp,wind,direction"]
temp_df                         = stream_df_yesterday['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True)
stream_df_yesterday["Datetime"] = temp_df[0] 
stream_df_yesterday['temp']     = temp_df[1] 
stream_df_yesterday['wind']     = temp_df[2]
stream_df_yesterday['direction'] = temp_df[3]
stream_df_yesterday = stream_df_yesterday.drop(columns=['Datetime,temp,wind,direction',
                                                        'direction','Datetime',
                                                        'wind'],axis=1)
stream_df_yesterday = stream_df_yesterday.reset_index(drop=True)
alberta_weather_merged.iloc[0,2] = stream_df_yesterday.iloc[0,0]
alberta_weather_merged.iloc[0:2,3] = stream_df_yesterday.iloc[0:2,0]
alberta_weather_merged.iloc[0:3,4] = stream_df_yesterday.iloc[0:3,0]

alberta_weather_merged

2021-02-16 10:58:05.786833 Outputing stream /api/StreamData/242498?fromDate=2/15/2021&toDate=2/15/2021 res code 200


,temp_calgary,temp_edmonton,temp_calgary_lag1,temp_calgary_lag2,temp_calgary_lag3,temp_edmonton_lag1,temp_edmonton_lag2,temp_edmonton_lag3
0,-20,-20,-23,-23,-23,NaN,NaN,NaN
1,-20,-20,-20,-24,-24,-20,NaN,NaN
2,-19,-20,-20,-20,-24,-20,-20,NaN
3,-18,-20,-19,-20,-20,-20,-20,-20
4,-17,-20,-18,-19,-20,-20,-20,-20
5,-17,-22,-17,-18,-19,-20,-20,-20
6,-15,-21,-17,-17,-18,-22,-20,-20
7,-14,-21,-15,-17,-17,-21,-22,-20
8,-12,-20,-14,-15,-17,-21,-21,-22
9,-10,-19,-12,-14,-15,-20,-21,-21


In [7]:
#one-day in the past feature
datetime_pastday                = now_time + pd.Timedelta(days= -1)
fromDate                        = '{}/{}/{}'.format(datetime_pastday.month,datetime_pastday.day, datetime_pastday.year )
toDate                          = fromDate
stream_data                     = nrgStreamApi.GetStreamDataByStreamId([242497], fromDate, toDate, 'csv', '')        
STREAM_DATA                     = StringIO(stream_data)
stream_df_yesterday             = pd.read_csv(STREAM_DATA, sep=";")
stream_df_yesterday             = stream_df_yesterday.iloc[16:]
stream_df_yesterday.columns     = ["Datetime,temp,wind,direction"]
temp_df                         = stream_df_yesterday['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True)
stream_df_yesterday["Datetime"] = temp_df[0] 
stream_df_yesterday['temp']     = temp_df[1] 
stream_df_yesterday['wind']     = temp_df[2]
stream_df_yesterday['direction'] = temp_df[3]
stream_df_yesterday = stream_df_yesterday.drop(columns=['Datetime,temp,wind,direction',
                                                        'direction','Datetime',
                                                        'wind'],axis=1)
stream_df_yesterday = stream_df_yesterday.reset_index(drop=True)
alberta_weather_merged.iloc[0,5] = stream_df_yesterday.iloc[0,0]
alberta_weather_merged.iloc[0:2,6] = stream_df_yesterday.iloc[0:2,0]
alberta_weather_merged.iloc[0:3,7] = stream_df_yesterday.iloc[0:3,0]

alberta_weather_merged

2021-02-16 10:58:07.283137 Outputing stream /api/StreamData/242497?fromDate=2/15/2021&toDate=2/15/2021 res code 200


,temp_calgary,temp_edmonton,temp_calgary_lag1,temp_calgary_lag2,temp_calgary_lag3,temp_edmonton_lag1,temp_edmonton_lag2,temp_edmonton_lag3
0,-20,-20,-23,-23,-23,-23,-23,-23
1,-20,-20,-20,-24,-24,-20,-23,-23
2,-19,-20,-20,-20,-24,-20,-20,-23
3,-18,-20,-19,-20,-20,-20,-20,-20
4,-17,-20,-18,-19,-20,-20,-20,-20
5,-17,-22,-17,-18,-19,-20,-20,-20
6,-15,-21,-17,-17,-18,-22,-20,-20
7,-14,-21,-15,-17,-17,-21,-22,-20
8,-12,-20,-14,-15,-17,-21,-21,-22
9,-10,-19,-12,-14,-15,-20,-21,-21


In [8]:
#alberta_weather_merged = alberta_weather_merged.values.astype(np.float)

alberta_weather_merged = alberta_weather_merged.apply(pd.to_numeric, errors='coerce')
alberta_weather_merged

,temp_calgary,temp_edmonton,temp_calgary_lag1,temp_calgary_lag2,temp_calgary_lag3,temp_edmonton_lag1,temp_edmonton_lag2,temp_edmonton_lag3
0,-20,-20,-23,-23,-23,-23,-23,-23
1,-20,-20,-20,-24,-24,-20,-23,-23
2,-19,-20,-20,-20,-24,-20,-20,-23
3,-18,-20,-19,-20,-20,-20,-20,-20
4,-17,-20,-18,-19,-20,-20,-20,-20
5,-17,-22,-17,-18,-19,-20,-20,-20
6,-15,-21,-17,-17,-18,-22,-20,-20
7,-14,-21,-15,-17,-17,-21,-22,-20
8,-12,-20,-14,-15,-17,-21,-21,-22
9,-10,-19,-12,-14,-15,-20,-21,-21


In [9]:
alberta_weather_merged['temp_calgary_lag1_squared'] =  alberta_weather_merged['temp_calgary_lag1']**2
alberta_weather_merged['temp_calgary_lag2_squared'] =  alberta_weather_merged['temp_calgary_lag2']**2
alberta_weather_merged['temp_calgary_lag3_squared'] =  alberta_weather_merged['temp_calgary_lag3']**2

alberta_weather_merged['temp_calgary_lag1_cube'] =  alberta_weather_merged['temp_calgary']**3
alberta_weather_merged['temp_calgary_lag2_cube'] =  alberta_weather_merged['temp_calgary']**3
alberta_weather_merged['temp_calgary_lag3_cube'] =  alberta_weather_merged['temp_calgary']**3

alberta_weather_merged['temp_edmonton_lag1_squared'] =  alberta_weather_merged['temp_edmonton_lag1']**2
alberta_weather_merged['temp_edmonton_lag2_squared'] =  alberta_weather_merged['temp_edmonton_lag2']**2
alberta_weather_merged['temp_edmonton_lag3_squared'] =  alberta_weather_merged['temp_edmonton_lag3']**2

alberta_weather_merged['temp_edmonton_lag1_cube'] =  alberta_weather_merged['temp_edmonton']**3
alberta_weather_merged['temp_edmonton_lag2_cube'] =  alberta_weather_merged['temp_edmonton']**3
alberta_weather_merged['temp_edmonton_lag3_cube'] =  alberta_weather_merged['temp_edmonton']**3


reordered_columns = ['temp_calgary', 'temp_edmonton', 
       'temp_calgary_lag1',  'temp_calgary_lag2','temp_calgary_lag3', 
       'temp_calgary_lag1_squared',  'temp_calgary_lag2_squared', 'temp_calgary_lag3_squared',
       'temp_calgary_lag1_cube',    'temp_calgary_lag2_cube', 'temp_calgary_lag3_cube',    
       'temp_edmonton_lag1',         'temp_edmonton_lag2', 'temp_edmonton_lag3',        
       'temp_edmonton_lag1_squared', 'temp_edmonton_lag2_squared','temp_edmonton_lag3_squared',
       'temp_edmonton_lag1_cube',   'temp_edmonton_lag2_cube','temp_edmonton_lag3_cube']

alberta_weather_merged = pd.DataFrame(alberta_weather_merged, columns = reordered_columns )
alberta_weather_merged

,temp_calgary,temp_edmonton,temp_calgary_lag1,temp_calgary_lag2,temp_calgary_lag3,temp_calgary_lag1_squared,temp_calgary_lag2_squared,temp_calgary_lag3_squared,temp_calgary_lag1_cube,temp_calgary_lag2_cube,temp_calgary_lag3_cube,temp_edmonton_lag1,temp_edmonton_lag2,temp_edmonton_lag3,temp_edmonton_lag1_squared,temp_edmonton_lag2_squared,temp_edmonton_lag3_squared,temp_edmonton_lag1_cube,temp_edmonton_lag2_cube,temp_edmonton_lag3_cube
0,-20,-20,-23,-23,-23,529,529,529,-8000,-8000,-8000,-23,-23,-23,529,529,529,-8000,-8000,-8000
1,-20,-20,-20,-24,-24,400,576,576,-8000,-8000,-8000,-20,-23,-23,400,529,529,-8000,-8000,-8000
2,-19,-20,-20,-20,-24,400,400,576,-6859,-6859,-6859,-20,-20,-23,400,400,529,-8000,-8000,-8000
3,-18,-20,-19,-20,-20,361,400,400,-5832,-5832,-5832,-20,-20,-20,400,400,400,-8000,-8000,-8000
4,-17,-20,-18,-19,-20,324,361,400,-4913,-4913,-4913,-20,-20,-20,400,400,400,-8000,-8000,-8000
5,-17,-22,-17,-18,-19,289,324,361,-4913,-4913,-4913,-20,-20,-20,400,400,400,-10648,-10648,-10648
6,-15,-21,-17,-17,-18,289,289,324,-3375,-3375,-3375,-22,-20,-20,484,400,400,-9261,-9261,-9261
7,-14,-21,-15,-17,-17,225,289,289,-2744,-2744,-2744,-21,-22,-20,441,484,400,-9261,-9261,-9261
8,-12,-20,-14,-15,-17,196,225,289,-1728,-1728,-1728,-21,-21,-22,441,441,484,-8000,-8000,-8000
9,-10,-19,-12,-14,-15,144,196,225,-1000,-1000,-1000,-20,-21,-21,400,441,441,-6859,-6859,-6859


In [10]:
prediction_df.iloc[:,13:33] = alberta_weather_merged.values
prediction_df

,hour_of_day,day,hour_x_day,weekend,month,year,AIL_24h_lagged,AIL_2day_lagged,AIL_3day_lagged,AIL_4day_lagged,...,hour_temp_edmonton_lag3,month_temp_edmonton_lag1,month_temp_edmonton_lag2,month_temp_edmonton_lag3,hour_temp_edmonton_lag1_sqd,hour_temp_edmonton_lag2_sqd,hour_temp_edmonton_lag3_sqd,month_temp_edmonton_lag1_cube,month_temp_edmonton_lag2_cube,month_temp_edmonton_lag3_cube
0,0,16,0,0,2,2021,10392.0,10546.0,10370.0,10668.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,16,16,0,2,2021,10422.0,10499.0,10344.0,10569.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,16,32,0,2,2021,10354.0,10461.0,10281.0,10478.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,16,48,0,2,2021,10342.0,10450.0,10177.0,10419.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,16,64,0,2,2021,10372.0,10444.0,10228.0,10467.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,16,80,0,2,2021,10444.0,10488.0,10193.0,10594.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,16,96,0,2,2021,10564.0,10600.0,10320.0,10810.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,16,112,0,2,2021,10750.0,10699.0,10626.0,11209.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,16,128,0,2,2021,10853.0,10807.0,10824.0,11343.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,16,144,0,2,2021,10978.0,10901.0,10992.0,11251.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
prediction_df.iloc[:,13:33]

,temp_calgary,temp_edmonton,temp_calgary_lag1,temp_calgary_lag2,temp_calgary_lag3,temp_calgary_lag1_squared,temp_calgary_lag2_squared,temp_calgary_lag3_squared,temp_calgary_lag1_cube,temp_calgary_lag2_cube,temp_calgary_lag3_cube,temp_edmonton_lag1,temp_edmonton_lag2,temp_edmonton_lag3,temp_edmonton_lag1_squared,temp_edmonton_lag2_squared,temp_edmonton_lag3_squared,temp_edmonton_lag1_cube,temp_edmonton_lag2_cube,temp_edmonton_lag3_cube
0,-20,-20,-23,-23,-23,529,529,529,-8000,-8000,-8000,-23,-23,-23,529,529,529,-8000,-8000,-8000
1,-20,-20,-20,-24,-24,400,576,576,-8000,-8000,-8000,-20,-23,-23,400,529,529,-8000,-8000,-8000
2,-19,-20,-20,-20,-24,400,400,576,-6859,-6859,-6859,-20,-20,-23,400,400,529,-8000,-8000,-8000
3,-18,-20,-19,-20,-20,361,400,400,-5832,-5832,-5832,-20,-20,-20,400,400,400,-8000,-8000,-8000
4,-17,-20,-18,-19,-20,324,361,400,-4913,-4913,-4913,-20,-20,-20,400,400,400,-8000,-8000,-8000
5,-17,-22,-17,-18,-19,289,324,361,-4913,-4913,-4913,-20,-20,-20,400,400,400,-10648,-10648,-10648
6,-15,-21,-17,-17,-18,289,289,324,-3375,-3375,-3375,-22,-20,-20,484,400,400,-9261,-9261,-9261
7,-14,-21,-15,-17,-17,225,289,289,-2744,-2744,-2744,-21,-22,-20,441,484,400,-9261,-9261,-9261
8,-12,-20,-14,-15,-17,196,225,289,-1728,-1728,-1728,-21,-21,-22,441,441,484,-8000,-8000,-8000
9,-10,-19,-12,-14,-15,144,196,225,-1000,-1000,-1000,-20,-21,-21,400,441,441,-6859,-6859,-6859


In [12]:
prediction_df['hour_temp_calgary']      = prediction_df['hour_of_day'] * prediction_df['temp_calgary']
prediction_df['month_temp_calgary']     = prediction_df['month'] * prediction_df['temp_calgary']

prediction_df['hour_temp_calgary_lag1'] = prediction_df['hour_of_day']* prediction_df['temp_calgary_lag1'] 
prediction_df['hour_temp_calgary_lag2'] = prediction_df['hour_of_day']* prediction_df['temp_calgary_lag2'] 
prediction_df['hour_temp_calgary_lag3'] = prediction_df['hour_of_day']* prediction_df['temp_calgary_lag3']

prediction_df['month_temp_calgary_lag1'] = prediction_df['month']* prediction_df['temp_calgary_lag1'] 
prediction_df['month_temp_calgary_lag2'] = prediction_df['month']* prediction_df['temp_calgary_lag2'] 
prediction_df['month_temp_calgary_lag3'] = prediction_df['month']* prediction_df['temp_calgary_lag3']

prediction_df['hour_temp_calgary_lag1_sqd'] = prediction_df['hour_of_day']* prediction_df['temp_calgary_lag1_squared'] 
prediction_df['hour_temp_calgary_lag2_sqd'] = prediction_df['hour_of_day']* prediction_df['temp_calgary_lag2_squared'] 
prediction_df['hour_temp_calgary_lag3_sqd'] = prediction_df['hour_of_day']* prediction_df['temp_calgary_lag3_squared']

prediction_df['month_temp_calgary_lag1_cube'] = prediction_df['month']* prediction_df['temp_calgary_lag1_squared'] 
prediction_df['month_temp_calgary_lag2_cube'] = prediction_df['month']* prediction_df['temp_calgary_lag2_squared'] 
prediction_df['month_temp_calgary_lag3_cube'] = prediction_df['month']* prediction_df['temp_calgary_lag3_squared']

#-----------
prediction_df['hour_temp_edmonton']  = prediction_df['hour_of_day'] * prediction_df['temp_edmonton']
prediction_df['month_temp_edmonton'] = prediction_df['month'] * prediction_df['temp_edmonton']

prediction_df['hour_temp_edmonton_lag1'] = prediction_df['hour_of_day']* prediction_df['temp_edmonton_lag1'] 
prediction_df['hour_temp_edmonton_lag2'] = prediction_df['hour_of_day']* prediction_df['temp_edmonton_lag2'] 
prediction_df['hour_temp_edmonton_lag3'] = prediction_df['hour_of_day']* prediction_df['temp_edmonton_lag3']

prediction_df['month_temp_edmonton_lag1'] = prediction_df['month']* prediction_df['temp_edmonton_lag1'] 
prediction_df['month_temp_edmonton_lag2'] = prediction_df['month']* prediction_df['temp_edmonton_lag2'] 
prediction_df['month_temp_edmonton_lag3'] = prediction_df['month']* prediction_df['temp_edmonton_lag3']

prediction_df['hour_temp_edmonton_lag1_sqd'] = prediction_df['hour_of_day']* prediction_df['temp_edmonton_lag1_squared'] 
prediction_df['hour_temp_edmonton_lag2_sqd'] = prediction_df['hour_of_day']* prediction_df['temp_edmonton_lag2_squared'] 
prediction_df['hour_temp_edmonton_lag3_sqd'] = prediction_df['hour_of_day']* prediction_df['temp_edmonton_lag3_squared']

prediction_df['month_temp_edmonton_lag1_cube'] = prediction_df['month']* prediction_df['temp_edmonton_lag1_squared'] 
prediction_df['month_temp_edmonton_lag2_cube'] = prediction_df['month']* prediction_df['temp_edmonton_lag2_squared'] 
prediction_df['month_temp_edmonton_lag3_cube'] = prediction_df['month']* prediction_df['temp_edmonton_lag3_squared']

In [13]:
prediction_df.to_csv('input_data_{}_{}_{}_generate_date_{}_v9_only_temp.csv'.format(day_of_run, month_of_run, year_of_run, now_time))

In [14]:
#Generate the forecast

#loaded_model = joblib.load('model_generated_on_13_2_2021_v6_non_holiday.joblib.dat')
loaded_model = joblib.load("model_generated_on_14_2_2021_v9_non_holiday_mape_loss_only_temp.joblib.dat")


forecast = pd.DataFrame() # this dataframe will contain the forecasts (timestamp and values)
forecast['datetime_of_forecast']= forecast_interval_stamps.values 
forecast['values']              = 0 #initializing the column with zero

#prediction_df.iloc[0,prediction_df.columns.get_loc('AIL_previous_hour')] = prediction_df.iloc[23,prediction_df.columns.get_loc('AIL_24h_lagged')] # assignment of AIL_previous_hour (24th hour of previous day)
#input_data               = prediction_df.iloc[0] # preparing input data
#predictions              = loaded_model.predict(np.array(input_data).reshape((1,-1))) # temporary location for forecast value
#prediction_df.iloc[1,prediction_df.columns.get_loc('AIL_previous_hour')] = predictions[0] #load forecast in AIL_previous_hour
#forecast.iloc[0,1]       = predictions[0]

for i in range(0, 24):
    input_data          = prediction_df.iloc[i]
    predictions         = loaded_model.predict(np.array(input_data).reshape((1,-1)))
    forecast.iloc[i,1]  = predictions[0]
#    if i+1 <= 23:
#        prediction_df.iloc[i+1,prediction_df.columns.get_loc('AIL_previous_hour')] = predictions[0]

forecast
#forecast.to_csv('forecast_{}_{}_{}_generate_date_{}.csv'.format(day_of_run, month_of_run, year_of_run, now_time ))

,datetime_of_forecast,values
0,2021-02-16 00:00:00,10522.670898
1,2021-02-16 01:00:00,10525.096680
2,2021-02-16 02:00:00,10425.216797
3,2021-02-16 03:00:00,10443.156250
4,2021-02-16 04:00:00,10464.999023
5,2021-02-16 05:00:00,10630.517578
6,2021-02-16 06:00:00,10734.660156
7,2021-02-16 07:00:00,10955.438477
8,2021-02-16 08:00:00,11092.630859
9,2021-02-16 09:00:00,11135.956055


In [15]:
'''
datetime_of_forecast	values
0	2021-02-14 00:00:00	10466.520508
1	2021-02-14 01:00:00	10438.880859
2	2021-02-14 02:00:00	10402.455078
3	2021-02-14 03:00:00	10428.241211
4	2021-02-14 04:00:00	10550.427734
5	2021-02-14 05:00:00	10511.577148
6	2021-02-14 06:00:00	10533.135742
7	2021-02-14 07:00:00	10722.361328
8	2021-02-14 08:00:00	10906.462891
9	2021-02-14 09:00:00	11113.807617
10	2021-02-14 10:00:00	11054.803711
11	2021-02-14 11:00:00	11118.104492
12	2021-02-14 12:00:00	11045.169922
13	2021-02-14 13:00:00	10944.636719
14	2021-02-14 14:00:00	10860.379883
15	2021-02-14 15:00:00	10931.954102
16	2021-02-14 16:00:00	11139.295898
17	2021-02-14 17:00:00	11391.276367
18	2021-02-14 18:00:00	11468.875977
19	2021-02-14 19:00:00	11430.413086
20	2021-02-14 20:00:00	11289.504883
21	2021-02-14 21:00:00	11146.918945
22	2021-02-14 22:00:00	10916.745117
23	2021-02-14 23:00:00	10707.790039




datetime_of_forecast	values
0	2021-02-14 00:00:00	10450.235352
1	2021-02-14 01:00:00	10417.850586
2	2021-02-14 02:00:00	10414.645508
3	2021-02-14 03:00:00	10521.308594
4	2021-02-14 04:00:00	10401.457031
5	2021-02-14 05:00:00	10389.750000
6	2021-02-14 06:00:00	10536.349609
7	2021-02-14 07:00:00	10759.925781
8	2021-02-14 08:00:00	10851.803711
9	2021-02-14 09:00:00	11015.231445
10	2021-02-14 10:00:00	11117.164062
11	2021-02-14 11:00:00	11227.749023
12	2021-02-14 12:00:00	11118.197266
13	2021-02-14 13:00:00	11031.517578
14	2021-02-14 14:00:00	10946.366211
15	2021-02-14 15:00:00	11008.471680
16	2021-02-14 16:00:00	11219.875977
17	2021-02-14 17:00:00	11381.377930
18	2021-02-14 18:00:00	11383.379883
19	2021-02-14 19:00:00	11365.541016
20	2021-02-14 20:00:00	11356.795898
21	2021-02-14 21:00:00	11250.417969
22	2021-02-14 22:00:00	10993.984375
23	2021-02-14 23:00:00	10685.560547


datetime_of_forecast	values
0	2021-02-14 00:00:00	10619.396484
1	2021-02-14 01:00:00	10527.563477
2	2021-02-14 02:00:00	10498.200195
3	2021-02-14 03:00:00	10499.145508
4	2021-02-14 04:00:00	10675.842773
5	2021-02-14 05:00:00	10667.277344
6	2021-02-14 06:00:00	10831.541992
7	2021-02-14 07:00:00	10855.270508
8	2021-02-14 08:00:00	10972.903320
9	2021-02-14 09:00:00	11059.010742
10	2021-02-14 10:00:00	11046.155273
11	2021-02-14 11:00:00	11032.979492
12	2021-02-14 12:00:00	11032.975586
13	2021-02-14 13:00:00	11025.170898
14	2021-02-14 14:00:00	10999.747070
15	2021-02-14 15:00:00	10988.877930
16	2021-02-14 16:00:00	11097.058594
17	2021-02-14 17:00:00	11231.550781
18	2021-02-14 18:00:00	11308.552734
19	2021-02-14 19:00:00	11303.566406
20	2021-02-14 20:00:00	11208.306641
21	2021-02-14 21:00:00	11071.291016
22	2021-02-14 22:00:00	10965.165039
23	2021-02-14 23:00:00	10684.395508

'''



'\ndatetime_of_forecast\tvalues\n0\t2021-02-14 00:00:00\t10466.520508\n1\t2021-02-14 01:00:00\t10438.880859\n2\t2021-02-14 02:00:00\t10402.455078\n3\t2021-02-14 03:00:00\t10428.241211\n4\t2021-02-14 04:00:00\t10550.427734\n5\t2021-02-14 05:00:00\t10511.577148\n6\t2021-02-14 06:00:00\t10533.135742\n7\t2021-02-14 07:00:00\t10722.361328\n8\t2021-02-14 08:00:00\t10906.462891\n9\t2021-02-14 09:00:00\t11113.807617\n10\t2021-02-14 10:00:00\t11054.803711\n11\t2021-02-14 11:00:00\t11118.104492\n12\t2021-02-14 12:00:00\t11045.169922\n13\t2021-02-14 13:00:00\t10944.636719\n14\t2021-02-14 14:00:00\t10860.379883\n15\t2021-02-14 15:00:00\t10931.954102\n16\t2021-02-14 16:00:00\t11139.295898\n17\t2021-02-14 17:00:00\t11391.276367\n18\t2021-02-14 18:00:00\t11468.875977\n19\t2021-02-14 19:00:00\t11430.413086\n20\t2021-02-14 20:00:00\t11289.504883\n21\t2021-02-14 21:00:00\t11146.918945\n22\t2021-02-14 22:00:00\t10916.745117\n23\t2021-02-14 23:00:00\t10707.790039\n\n\n\n\ndatetime_of_forecast\tvalues\n0\t